# Data Cleaning

In [57]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

## Berlin

In [301]:
berlin = pd.read_csv('./Berlin_Results/Original/Berlin_Marathon_data_1974_2019_alt.csv')

def berlin_age(x):
    if x in [i for i in range(0,18)]:
        return np.nan
    elif x in [i for i in range(18,40)]:
        return '18-39'
    elif x in [i for i in range(40,45)]:
        return '40-44'
    elif x in [i for i in range(45,50)]:
        return '45-49'
    elif x in [i for i in range(50,55)]:
        return '50-54'
    elif x in [i for i in range(55,60)]:
        return '55-59'
    elif x in [i for i in range(45,65)]:
        return '60-64'
    elif x in [i for i in range(65,70)]:
        return '65-69'
    elif x in [i for i in range(70,99)]:
        return '70+'
    return x


# binarize gender
berlin['male'] = np.where(berlin['GENDER'] == 'male', 1, 0)


# drop rows with no time
berlin = pd.DataFrame(berlin[berlin['TIME'] != 'no time'])

# split time into hours, minutes, seconds
berlin['time_h'] = berlin['TIME'].str[0].astype(int)
berlin['time_m'] = berlin['TIME'].str[2:4].astype(int)
berlin['time_s'] = berlin['TIME'].str[5:].astype(int)


# combine split times into total seconds
berlin['time_seconds'] = berlin['time_s'] + (berlin['time_m'] * 60) + (berlin['time_h'] * 60 * 60)


# drop unnecessary columns
berlin.drop(columns=['COUNTRY', 'TIME', 'GENDER', 'time_h', 'time_m', 'time_s'], inplace=True)

# adjust column names
berlin.columns = ['year', 'age', 'male', 'time_seconds']

berlin['age'] = berlin['age'].apply(lambda x: berlin_age(x))

# reduce size
berlin['year'] = pd.to_numeric(berlin['year'], downcast='unsigned')
berlin['male'] = pd.to_numeric(berlin['male'], downcast='unsigned')
berlin['time_seconds'] = pd.to_numeric(berlin['time_seconds'], downcast='unsigned')

# save cleaned sets to csv
berlin.to_csv('./Berlin_Results/Clean/Clean_Berlin_Results_1974_2019.csv', index=False)

## Boston

In [265]:
boston_dict = {file[:-4]: pd.read_csv(f'./Boston_Results/Original/{file}') for file in os.listdir('./Boston_Results/Original/')}

def boston_age(x):
    if x in [i for i in range(0,18)]:
        return np.nan
    elif x in [i for i in range(18,40)]:
        return '18-39'
    elif x in [i for i in range(40,45)]:
        return '40-44'
    elif x in [i for i in range(45,50)]:
        return '45-49'
    elif x in [i for i in range(50,55)]:
        return '50-54'
    elif x in [i for i in range(55,60)]:
        return '55-59'
    elif x in [i for i in range(45,65)]:
        return '60-64'
    elif x in [i for i in range(65,70)]:
        return '65-69'
    elif x in [i for i in range(70,99)]:
        return '70+'
    return x


combined_df = pd.DataFrame()
for file in boston_dict:
    combined_df = pd.concat([combined_df, boston_dict[file]], ignore_index=True)
    
combined_df.drop(columns=['bib', 'name', 'city', 'state', 'country',
       'blank', 'overall_place', 'gender_place', 'division_place',
       'net_time'], inplace=True)

combined_df['age'] = combined_df['age'].apply(lambda x: boston_age(x))
combined_df['male'] = np.where(combined_df['gender'] == 'M', 1, 0)
combined_df['time_seconds'] = combined_df['official_time'].str[5:].astype(int) + (combined_df['official_time'].str[2:4].astype(int) * 60) + (combined_df['official_time'].str[0].astype(int) * 60 * 60)

combined_df['year'] = pd.to_numeric(combined_df['year'], downcast='unsigned')
combined_df['time_seconds'] = pd.to_numeric(combined_df['time_seconds'], downcast='unsigned')
combined_df['male'] = pd.to_numeric(combined_df['male'], downcast='unsigned')

combined_df.drop(columns=['official_time', 'gender'], inplace=True)

combined_df.to_csv('./Boston_Results/Clean/Clean_Boston_Results_Combined.csv', index=False)

## Chicago

In [238]:
chicago_dict = {file[16:-4]: pd.read_csv(f'./Chicago_Results/Original/{file}') for file in os.listdir('./Chicago_Results/Original/')}

def chicago_age(x):
    try:
        x = int(x[:2])
        if x in [i for i in range(0,18)]:
            return np.nan
        elif x in [i for i in range(18,40)]:
            return '18-39'
        elif x in [i for i in range(40,45)]:
            return '40-44'
        elif x in [i for i in range(45,50)]:
            return '45-49'
        elif x in [i for i in range(50,55)]:
            return '50-54'
        elif x in [i for i in range(55,60)]:
            return '55-59'
        elif x in [i for i in range(45,65)]:
            return '60-64'
        elif x in [i for i in range(65,70)]:
            return '65-69'
        elif x in [i for i in range(70,99)]:
            return '70+'
        return x
    except:
        return np.nan

combined_df = pd.DataFrame()
for file in chicago_dict:
    combined_df = pd.concat([combined_df, chicago_dict[file]], ignore_index=True)
    
combined_df['age'] = combined_df['age'].apply(lambda x: chicago_age(x))
combined_df.dropna(inplace=True)

combined_df['male'] = np.where(combined_df['gender'] == 'Men', 1, 0)

combined_df = pd.DataFrame(combined_df[combined_df['time'].str[:2] != '00'])
combined_df['time_seconds'] = ((combined_df['time'].str[6:].astype(int)) + (combined_df['time'].str[3:5].astype(int) * 60) + (combined_df['time'].str[:2].astype(int) * 60 * 60))

combined_df['year'] = pd.to_numeric(combined_df['year'], downcast='unsigned')
combined_df['male'] = pd.to_numeric(combined_df['male'], downcast='unsigned')
combined_df['time_seconds'] = pd.to_numeric(combined_df['time_seconds'], downcast='unsigned')

combined_df = combined_df[['year', 'age', 'male', 'time_seconds']]

combined_df.to_csv('./Chicago_Results/Clean/Clean_Chicago_Results_Combined.csv', index=False)

## London

In [213]:
def age_range(x):    
    x = str(x)
    if x in [str(i) for i in range(0,18)]:
        return np.nan
    elif x in [str(i) for i in range(18,40)]:
        return '18-39'
    elif x in [str(i) for i in range(40,45)]:
        return '40-44'
    elif x in [str(i) for i in range(45,50)]:
        return '45-49'
    elif x in [str(i) for i in range(50,55)]:
        return '50-54'
    elif x in [str(i) for i in range(55,60)]:
        return '55-59'
    elif x in [str(i) for i in range(45,65)]:
        return '60-64'
    elif x in [str(i) for i in range(65,70)]:
        return '65-69'
    elif x in [str(i) for i in range(70,99)]:
        return '70+'
    elif x in ['ELITE', 'Elite', 'Elite18-39', 'elite', '18-34', 'Open','20-24', '30-34', '25-29', '35-39', '15-19', '18-39', '18-19', '20-29', '30-39']:
        return '18-39'
    elif x in ['70-99', '75-79', '80-99', '80-84']:
        return '70+'
    elif x in ['EliteMasters', '', 'T42', 'T11-T12', 'T44-T46', 'nan']:
         return np.nan    
    elif x in ['Elite40-44']:
        return '40-44'
    elif x in ['Elite45-49']:
        return '45-49'
    return x

london_dict = {file[:-4]: pd.read_csv(f'./London_Results/Original/{file}') for file in os.listdir('./London_Results/Original/')}

combined_df = pd.DataFrame(columns=['year', 'age', 'male', 'time_seconds'])

for file in london_dict:
    
    if (file == 'London_Results_2001' or file == 'London_Results_2002'):
        london_dict[file] = london_dict[file][['Last Name, First Name(Sex/Age)', 'Time']]
        london_dict[file]['age'] = london_dict[file]['Last Name, First Name(Sex/Age)'].str[-3:-1]
        london_dict[file]['male'] = np.where(london_dict[file]['Last Name, First Name(Sex/Age)'].str[-4] == 'M', 1, 0)
    
    elif file == 'London_Results_2017':
        london_dict[file] = london_dict[file][['DIV', 'Net Time']]
        london_dict[file]['age'] = london_dict[file]['DIV'].str[1:]
        london_dict[file]['male'] = np.where(london_dict[file]['DIV'].str[0] == 'M', 1, 0)
                
    else:
        london_dict[file] = london_dict[file][['DIV', 'Time']]
        london_dict[file]['age'] = london_dict[file]['DIV'].str[1:]
        london_dict[file]['male'] = np.where(london_dict[file]['DIV'].str[0] == 'M', 1, 0)
    
    
    london_dict[file].columns = ['group', 'time', 'age', 'male']
    london_dict[file]['time_seconds'] = ((london_dict[file]['time'].str[5:].astype(int)) + (london_dict[file]['time'].str[2:4].astype(int) * 60) + (london_dict[file]['time'].str[0].astype(int) * 60 * 60))
    london_dict[file]['year'] = file[-4:]
    
    london_dict[file] = london_dict[file][['year', 'age', 'male', 'time_seconds']]
    
    combined_df = pd.concat([combined_df, london_dict[file]], ignore_index=True)
    
combined_df['age'] = combined_df['age'].apply(lambda x: age_range(x))

combined_df.dropna(inplace=True)

combined_df['year'] = pd.to_numeric(combined_df['year'], downcast='unsigned')
combined_df['male'] = pd.to_numeric(combined_df['male'], downcast='unsigned')
combined_df['time_seconds'] = pd.to_numeric(combined_df['time_seconds'], downcast='unsigned')

combined_df.to_csv('./London_Results/Clean/Clean_London_Results_Combined.csv', index=False)

## NYC

In [299]:
nyc_dict = {file[:-4]: pd.read_csv(f'./NYC_Results/Original/{file}') for file in os.listdir('./NYC_Results/Original/')}

def nyc_age(x):
    try:
        x = int(x[:2])
        if x in [i for i in range(0,18)]:
            return np.nan
        elif x in [i for i in range(18,40)]:
            return '18-39'
        elif x in [i for i in range(40,45)]:
            return '40-44'
        elif x in [i for i in range(45,50)]:
            return '45-49'
        elif x in [i for i in range(50,55)]:
            return '50-54'
        elif x in [i for i in range(55,60)]:
            return '55-59'
        elif x in [i for i in range(45,65)]:
            return '60-64'
        elif x in [i for i in range(65,70)]:
            return '65-69'
        elif x in [i for i in range(70,99)]:
            return '70+'
        return x
    except:
        return np.nan

combined_df = pd.DataFrame()

for file in nyc_dict:
    nyc_dict[file]['year'] = file[-4:]
    combined_df = pd.concat([combined_df, nyc_dict[file]], ignore_index=True)

combined_df['age'] = combined_df['groups'].str[1:]    
combined_df['age'] = combined_df['age'].apply(lambda x: nyc_age(x))

combined_df['male'] = np.where(combined_df['groups'].str[0] == 'M', 1, 0)
                               
combined_df['time_seconds'] = combined_df['times'].str[5:].astype(int) + (combined_df['times'].str[2:4].astype(int) * 60) + (combined_df['times'].str[0].astype(int) * 60 * 60)


combined_df = combined_df[['year', 'age', 'male', 'time_seconds']]

combined_df['year'] = pd.to_numeric(combined_df['year'], downcast='unsigned')
combined_df['male'] = pd.to_numeric(combined_df['male'], downcast='unsigned')
combined_df['time_seconds'] = pd.to_numeric(combined_df['time_seconds'], downcast='unsigned')

combined_df.dropna(inplace=True)

combined_df.to_csv('./NYC_Results/Clean/Clean_NYC_Results_Combined.csv', index=False)

## Weather

In [197]:
# read in weather files
berlin_weather = pd.read_csv('./Berlin_Results/Original/Berlin_Weather.csv')
boston_weather = pd.read_csv('./Boston_Results/Original/Boston_Weather.csv')
chicago_weather = pd.read_csv('./Chicago_Results/Original/Chicago_Weather.csv')
london_weather = pd.read_csv('./London_Results/Original/London_Weather.csv')
nyc_weather = pd.read_csv('./NYC_Results/Original/NYC_Weather.csv')

cv = CountVectorizer(tokenizer=lambda x: x.split(', '))
conditions = pd.DataFrame(pd.concat([berlin_weather, boston_weather, chicago_weather, london_weather, nyc_weather], ignore_index=True)['Conditions'])
cv.fit(conditions['Conditions'])



def clean_vectorize(dataframe, cv):

    dataframe['year'] = dataframe['Date time'].str[-4:]
    dataframe.drop(columns=['Address', 'Heat Index', 'Wind Gust', 'Wind Direction', 'Wind Chill',
                            'Snow Depth', 'Sea Level Pressure', 'Weather Type', 'Latitude', 'Longitude',
                            'Resolved Address', 'Name', 'Info', 'Date time', 'Dew Point', 'Visibility'], inplace=True)
    dataframe = dataframe.replace(np.nan, 100) # only missing cloud cover data for berlin weather on a rainy day. assuming it is 100%

    conditions = pd.DataFrame.sparse.from_spmatrix(cv.transform(dataframe['Conditions']), columns=cv.get_feature_names_out())
    dataframe = dataframe.merge(conditions, how='left', left_index=True, right_index=True).drop(columns='Conditions')

    dataframe.columns = [col.lower().strip().replace(' ', '_') for col in dataframe.columns]
    
    for col in dataframe.columns:
        try:
            dataframe[col] = pd.to_numeric(dataframe[col], downcast='unsigned')
        except:
            continue
    
    return dataframe




berlin_weather = clean_vectorize(berlin_weather, cv)
boston_weather = clean_vectorize(boston_weather, cv)
chicago_weather = clean_vectorize(chicago_weather, cv)
london_weather = clean_vectorize(london_weather, cv)
nyc_weather = clean_vectorize(nyc_weather, cv)

C:\Users\David\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [198]:
berlin_results = pd.read_csv('./Berlin_Results/Clean/Clean_Berlin_Results_1974_2019.csv')
boston_results = pd.read_csv('./Boston_Results/Clean/Clean_Boston_Results_Combined.csv')
chicago_results = pd.read_csv('./Chicago_Results/Clean/Clean_Chicago_Results_Combined.csv')
london_results = pd.read_csv('./London_Results/Clean/Clean_London_Results_Combined.csv')
nyc_results = pd.read_csv('./NYC_Results/Clean/Clean_NYC_Results_Combined.csv')

berlin_combined = berlin_results.merge(berlin_weather, how='left', on='year')
boston_combined = boston_results.merge(boston_weather, how='left', on='year')
chicago_combined = chicago_results.merge(chicago_weather, how='left', on='year')
london_combined = london_results.merge(london_weather, how='left', on='year')
nyc_combined = nyc_results.merge(nyc_weather, how='left', on='year')

berlin_combined.to_csv('./Berlin_Results/Clean/Clean_Berlin_Results_Weather.csv', index=False)
boston_combined.to_csv('./Boston_Results/Clean/Clean_Boston_Results_Weather.csv', index=False)
chicago_combined.to_csv('./Chicago_Results/Clean/Clean_Chicago_Results_Weather.csv', index=False)
london_combined.to_csv('./London_Results/Clean/Clean_London_Results_Weather.csv', index=False)
nyc_combined.to_csv('./NYC_Results/Clean/Clean_NYC_Results_Weather.csv', index=False)